# Identificação de *Motifs*

In [1]:
import docker, os

a = %pwd
wd = (a.rsplit('/',2))[0]

GENE = "ITGAL"
BLAST_Results_ncbi=os.path.join("homologue", GENE, "NM_001114380_ITGAL.fasta")
BLAST_Results_uniprot=os.path.join("homologue", GENE, "B20210131A94466D2655679D1FD8953E075198DA8030E49B.fasta")
REF=os.path.join("reference", GENE, "ITGAL_prot.fasta")
D_MEME='quay.io/biocontainers/meme:5.3.0--py37pl526h072abfd_0'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [2]:
client = docker.from_env()
bwa_docker_cont = client.containers.run(
    image=D_MEME,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/motif/',
    entrypoint="bash",
    command='{0}/scripts_bash/meme.sh {1} {2} {3} {4}'.format(WORK_CONT_DIRECTORY, BLAST_Results_ncbi, BLAST_Results_uniprot, REF, GENE))

In [2]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "meme/meme.xml")) as file:
    meme_record = motifs.parse(file, "MEME")

for motif in meme_record:
    print(motif.id, ":" ,motif.name, motif.num_occurrences)

motif_1 : WRPKQEVRGTQIGSYFGASLCSVDVDRDGSTDLLLIGAPHYYEZTRGGQV 13
motif_2 : GRFGAAJTVLGDVNGDKLTDVAIGAPGEEENRGAV 13
motif_3 : PSQRITGSQLSPGLQYFGQSJSGGQDLTQDGLVDLAVGAZGHVLLLRSRP 13
motif_4 : VKNPLPLIVGSSIGGLLLLALITAALYKLGFFKRQYKEMME 13
motif_5 : EAVNMSLGLSLATDTNDSQLLACGPTLSRTCGENTYLTGLC 13


A utilização do MEME para anotação de novo resultou em 5 potências motif, os motifs apresentam-se conservadados em todas as sequências homólogas. Estes não coincidem com nenhuma das anotações, dominios ou features dos ficheiros Uniprot, apesar disto alguns encontram-se próximos ou contém multiplaas anotações de estruturas secundárias, ou pontos de glicolisação.

:::{figure,myclass}
![Figura 1 - NJ ITGAL](img/weblogo_ITGAL.png)

Weblogos dos motifs anotados *de novo*
:::

:::{figure,myclass}
![Figura 1 - NJ ITGAL](img/meme_loc_ITGAL.png)

Localização dos motifs anotados *de novo*
:::

In [3]:
from scripts_python.vcf_gff import gff2rec

motif_in_ref = gff2rec(os.path.join(wd,"data/motif", GENE, "fimo/fimo.gff"))
for seq in motif_in_ref:
    for feature in seq.features:
        print(feature)

type: sequence_motif
location: [361:411](+)
id: WRPKQEVRGTQIGSYFGASLCSVDVDRDGSTDLLLIGAPHYYEZTRGGQV-MEME-1-1-NP_001107852.1
qualifiers:
    Key: Alias, Value: ['MEME-1']
    Key: ID, Value: ['WRPKQEVRGTQIGSYFGASLCSVDVDRDGSTDLLLIGAPHYYEZTRGGQV-MEME-1-1-NP_001107852.1']
    Key: Name, Value: ['WRPKQEVRGTQIGSYFGASLCSVDVDRDGSTDLLLIGAPHYYEZTRGGQV_NP_001107852.1+']
    Key: pvalue, Value: ['1.87e-48']
    Key: qvalue, Value: ['1.92e-45']
    Key: score, Value: ['477']
    Key: sequence, Value: ['WSQVQTIHGTQIGSYFGGELCGVDVDQDGETELLLIGAPLFYGEQRGGRV']
    Key: source, Value: ['fimo']

type: sequence_motif
location: [435:470](+)
id: GRFGAAJTVLGDVNGDKLTDVAIGAPGEEENRGAV-MEME-2-1-NP_001107852.1
qualifiers:
    Key: Alias, Value: ['MEME-2']
    Key: ID, Value: ['GRFGAAJTVLGDVNGDKLTDVAIGAPGEEENRGAV-MEME-2-1-NP_001107852.1']
    Key: Name, Value: ['GRFGAAJTVLGDVNGDKLTDVAIGAPGEEENRGAV_NP_001107852.1+']
    Key: pvalue, Value: ['1.38e-33']
    Key: qvalue, Value: ['1.44e-30']
    Key: score, Value: ['329'

Tanto o motif "PSQRITGSQLSPGLQYFGQSJSGGQDLTQDGLVDLAVGAZGHVLLLRSRP" como o motif "GRFGAAJTVLGDVNGDKLTDVAIGAPGEEENRGAV", se encontram repetidos várias vezes.

:::{figure,myclass}
![Figura 1 - NJ ITGAL](img/mast_loc_ITGAL.png)

Exemplo das localização dos motifs anotados num segundo subset de sequências homólogas.
:::

Neste caso só foram reconhecidas 2 sequências no MAST devido ao threshold que este têm de valor 10 para o E-value das sequências.
Isto pode indicar que esta sequência não é especifica e como tal pode não ser de interesse alinhar/analizar as mesmas para motifs.